In [19]:
import pandas as pd
import sqlite3
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv("/kaggle/input/csv-data/Accident_Info.csv")
df.head()
df = df.loc[:, ~df.columns.str.contains("^Unnamed", na=False)]
df.head()
df = df.drop(columns=["Column3", "Column7"], errors="ignore")
df.head()
df = df.drop(columns=["Severity_Clean"], errors="ignore")
df = df.drop(columns=["Speed_Band"], errors="ignore")
df.info()
df.head()


/kaggle/input/csv-data/Accident_Info.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Accident Severity           999 non-null    object 
 1   Day of Week                 999 non-null    object 
 2   Light Conditions            999 non-null    object 
 3   Local Authority (District)  999 non-null    object 
 4   Number of Casualties        999 non-null    float64
 5   Number of Vehicles          999 non-null    float64
 6   Road Surface Conditions     999 non-null    object 
 7   Speed limit                 999 non-null    float64
 8   Time                        999 non-null    object 
 9   Year                        999 non-null    float64
 10  Hour                        1000 non-null   int64  
 11  Peak_Period                 1000 non-null   object 
 12  Weekday_or_Weekend          1000 non-null   object

,Accident Severity,Day of Week,Light Conditions,Local Authority (District),Number of Casualties,Number of Vehicles,Road Surface Conditions,Speed limit,Time,Year,Hour,Peak_Period,Weekday_or_Weekend
0,Slight,Monday,Darkness - lights lit,"Bristol, City of",1.0,2.0,Wet or damp,30.0,17:50,2005.0,17,Peak,Weekday
1,Serious,Saturday,Daylight,North Kesteven,1.0,1.0,Dry,30.0,14:45,2008.0,14,Off-Peak,Weekend
2,Slight,Friday,Daylight,Stockport,1.0,1.0,Dry,30.0,15:45,2006.0,15,Off-Peak,Weekday
3,Slight,Thursday,Daylight,Gloucester,1.0,2.0,Wet or damp,60.0,11:19,2005.0,11,Off-Peak,Weekday
4,Slight,Friday,Daylight,Breckland,1.0,1.0,Dry,30.0,13:38,2008.0,13,Off-Peak,Weekday


## Data cleaning

* input data
* removed unneccesary columns
* removed unnamed and not needed columns
* removed NaN values

In [21]:
essential_cols = [
    "Accident Severity", "Day of Week", "Light Conditions",
    "Local Authority (District)", "Number of Casualties",
    "Number of Vehicles", "Road Surface Conditions",
    "Speed limit", "Time", "Year", "Hour", "Peak_Period", "Weekday_or_Weekend"
]

df = df.dropna(subset=essential_cols)

# Keep only true severity categories
df = df[df["Accident Severity"].isin(["Slight", "Serious", "Fatal"])]

df.shape, df["Accident Severity"].value_counts()


((999, 13),
 Accident Severity
 Slight     838
 Serious    141
 Fatal       20
 Name: count, dtype: int64)

## Final cleaning checks

* dataset is mostly clean.  
* removed any remaining missing values in the key analysis columns and kept only valid severity categories.


In [23]:
pd.read_sql_query("""
SELECT
  Year,
  "Accident Severity" AS severity,
  "Local Authority (District)" AS local_authority,
  "Light Conditions" AS light_conditions,
  "Road Surface Conditions" AS road_surface,
  "Speed limit" AS speed_limit,
  "Number of Casualties" AS casualties
FROM Table3
WHERE "Accident Severity" = 'Serious'
ORDER BY Year DESC
LIMIT 20;
""", conn)


,Year,severity,local_authority,light_conditions,road_surface,speed_limit,casualties
0,2010.0,Serious,Medway,Darkness - lights lit,Dry,40.0,1.0
1,2010.0,Serious,South Northamptonshire,Daylight,Dry,60.0,1.0
2,2010.0,Serious,Lancaster,Daylight,Dry,50.0,2.0
3,2010.0,Serious,Cornwall,Darkness - lights lit,Wet or damp,40.0,1.0
4,2010.0,Serious,Lambeth,Darkness - lights lit,Dry,30.0,1.0
5,2010.0,Serious,Cornwall,Daylight,Wet or damp,60.0,2.0
6,2010.0,Serious,Elmbridge,Daylight,Dry,30.0,1.0
7,2010.0,Serious,Rossendale,Darkness - lights lit,Dry,30.0,1.0
8,2010.0,Serious,Redbridge,Darkness - lights lit,Dry,30.0,1.0
9,2010.0,Serious,Birmingham,Daylight,Dry,30.0,1.0


##Basic WHERE query (filter)
* show the most recent **Serious** accidents.


In [30]:
pd.read_sql_query("""
SELECT
  "Accident Severity" AS severity,
  COUNT(*) AS incidents,
  ROUND(
    100.0 * COUNT(*) / (SELECT COUNT(*) FROM Table3),
    2
  ) AS pct_of_total
FROM Table3
GROUP BY "Accident Severity"
ORDER BY incidents DESC;
""", conn)


,severity,incidents,pct_of_total
0,Slight,838,83.88
1,Serious,141,14.11
2,Fatal,20,2.00


## GROUP BY (severity distribution)

* Counts and % share by severity.


In [25]:
pd.read_sql_query("""
SELECT
  Peak_Period,
  COUNT(*) AS incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY Peak_Period
ORDER BY severe_rate DESC;
""", conn)


,Peak_Period,incidents,severe_incidents,severe_rate
0,Off-Peak,616,123,0.200
1,Peak,383,38,0.099


## KPI: Severe rate by Peak vs Off-Peak

Defines **severe** as Serious or Fatal and calculates a severe rate.


In [26]:
pd.read_sql_query("""
SELECT
  "Light Conditions" AS light_conditions,
  "Road Surface Conditions" AS road_surface,
  COUNT(*) AS incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY light_conditions, road_surface
HAVING incidents >= 20
ORDER BY severe_rate DESC, incidents DESC
LIMIT 15;
""", conn)


,light_conditions,road_surface,incidents,severe_incidents,severe_rate
0,Darkness - no lighting,Dry,28,10,0.357
1,Darkness - no lighting,Wet or damp,28,6,0.214
2,Darkness - lights lit,Wet or damp,68,14,0.206
3,Darkness - lights lit,Dry,102,20,0.196
4,Daylight,Dry,555,79,0.142
5,Daylight,Wet or damp,180,24,0.133


## Risk conditions: Light + Road Surface

Which combinations have the highest severe rate (with a minimum volume threshold).


In [29]:
pd.read_sql_query("""
SELECT
  CASE
    WHEN "Speed limit" <= 30 THEN '0–30'
    WHEN "Speed limit" <= 50 THEN '31–50'
    WHEN "Speed limit" <= 60 THEN '51–60'
    ELSE '61+'
  END AS speed_band,
  COUNT(*) AS incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY speed_band
ORDER BY Severe_rate DESC;
""", conn)


,speed_band,incidents,severe_incidents,severe_rate
0,31–50,99,20,0.202
1,51–60,184,37,0.201
2,61+,83,14,0.169
3,0–30,633,90,0.142


## Step 7 — Speed limit banding (CASE WHEN)

Create speed bands in SQL and compare severe rates by band.


In [31]:
pd.read_sql_query("""
SELECT
  Year,
  "Local Authority (District)" AS local_authority,
  "Light Conditions" AS light_conditions,
  "Road Surface Conditions" AS road_surface,
  "Speed limit" AS speed_limit,
  "Number of Casualties" AS casualties
FROM Table3
WHERE "Accident Severity" IN ('Serious','Fatal')
  AND "Light Conditions" LIKE 'Dark%'
  AND "Road Surface Conditions" IN ('Wet or damp','Frost or ice','Snow')
ORDER BY Year DESC
LIMIT 20;
""", conn)


,Year,local_authority,light_conditions,road_surface,speed_limit,casualties
0,2010.0,Cornwall,Darkness - lights lit,Wet or damp,40.0,1.0
1,2009.0,Bradford,Darkness - lights lit,Frost or ice,30.0,1.0
2,2009.0,Solihull,Darkness - lights lit,Wet or damp,40.0,1.0
3,2009.0,Surrey Heath,Darkness - no lighting,Frost or ice,60.0,4.0
4,2008.0,Hambleton,Darkness - no lighting,Frost or ice,70.0,2.0
5,2008.0,Harrogate,Darkness - no lighting,Wet or damp,60.0,5.0
6,2008.0,Inverclyde,Darkness - lights lit,Wet or damp,30.0,1.0
7,2008.0,Ryedale,Darkness - no lighting,Wet or damp,70.0,2.0
8,2008.0,Babergh,Darkness - lights unlit,Wet or damp,30.0,1.0
9,2008.0,Wirral,Darkness - lights lit,Wet or damp,30.0,4.0


## WHERE: High-risk accidents under poor conditions

This query focuses on serious and fatal accidents that occurred under higher-risk environmental conditions.
Filtering in this way helps identify scenarios where targeted interventions may be most effective.


In [33]:
severity_lookup = pd.DataFrame({
    "Accident Severity": ["Slight", "Serious", "Fatal"],
    "Severity_Group": ["Non-severe", "Severe", "Severe"]
})

severity_lookup.to_sql(
    "Severity_Lookup",
    conn,
    index=False,
    if_exists="replace"
)

severity_lookup
pd.read_sql_query("""
SELECT
  s.Severity_Group,
  t.Peak_Period,
  COUNT(*) AS incidents,
  ROUND(
    1.0 * COUNT(*) / SUM(COUNT(*)) OVER (),
    3
  ) AS proportion_of_total
FROM Table3 t
JOIN Severity_Lookup s
  ON t."Accident Severity" = s."Accident Severity"
GROUP BY s.Severity_Group, t.Peak_Period
ORDER BY s.Severity_Group, incidents DESC;
""", conn)


,Severity_Group,Peak_Period,incidents,proportion_of_total
0,Non-severe,Off-Peak,493,0.493
1,Non-severe,Peak,345,0.345
2,Severe,Off-Peak,123,0.123
3,Severe,Peak,38,0.038


## JOIN: Severity grouping using a lookup table

To demonstrate relational data modelling, a small lookup table is created to group accident severities into
`Severe` and `Non-severe` categories.

In [34]:
pd.read_sql_query("""
SELECT
  "Light Conditions" AS light_conditions,
  "Road Surface Conditions" AS road_surface,
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY light_conditions, road_surface
HAVING total_incidents >= 20
ORDER BY severe_rate DESC, total_incidents DESC
LIMIT 10;
""", conn)


,light_conditions,road_surface,total_incidents,severe_incidents,severe_rate
0,Darkness - no lighting,Dry,28,10,0.357
1,Darkness - no lighting,Wet or damp,28,6,0.214
2,Darkness - lights lit,Wet or damp,68,14,0.206
3,Darkness - lights lit,Dry,102,20,0.196
4,Daylight,Dry,555,79,0.142
5,Daylight,Wet or damp,180,24,0.133


## Insight: Most severe accident environments

This analysis identifies combinations of lighting and road surface conditions associated with the highest proportion of serious and fatal accidents.
Understanding these environments helps target safety interventions more effectively.


In [35]:
pd.read_sql_query("""
SELECT
  Hour,
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY Hour
HAVING total_incidents >= 10
ORDER BY severe_rate DESC;
""", conn)


,Hour,total_incidents,severe_incidents,severe_rate
0,0,10,4,0.400
1,23,27,10,0.370
2,1,12,4,0.333
3,10,43,14,0.326
4,3,12,3,0.250
5,13,73,16,0.219
6,22,24,5,0.208
7,19,63,13,0.206
8,21,25,5,0.200
9,20,30,6,0.200


## Insight: Severity by time of day

This analysis examines how accident severity varies by hour of day, highlighting time periods with a higher proportion of serious or fatal incidents.


In [36]:
pd.read_sql_query("""
SELECT
  Peak_Period,
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY Peak_Period
ORDER BY severe_rate DESC;
""", conn)


,Peak_Period,total_incidents,severe_incidents,severe_rate
0,Off-Peak,616,123,0.200
1,Peak,383,38,0.099


## Insight: Severity during peak vs off-peak periods

This analysis compares the proportion of serious and fatal accidents during peak and off-peak traffic periods.


In [40]:
pd.read_sql_query("""
SELECT
  "Speed limit" AS speed_limit,
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY speed_limit
HAVING total_incidents >= 20
ORDER BY total_incidents DESC;
""", conn)


,speed_limit,total_incidents,severe_incidents,severe_rate
0,30.0,624,88,0.141
1,60.0,184,37,0.201
2,70.0,83,14,0.169
3,40.0,71,15,0.211
4,50.0,28,5,0.179


## Insight: Severity by speed environment

This analysis compares accident severity across different speed limit environments to assess how speed context relates to accident outcomes.


In [41]:
pd.read_sql_query("""
SELECT
  "Day of Week" AS day_of_week,
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY "Day of Week"
HAVING total_incidents >= 20
ORDER BY severe_rate DESC;
""", conn)


,day_of_week,total_incidents,severe_incidents,severe_rate
0,Saturday,152,36,0.237
1,Sunday,109,24,0.220
2,Monday,150,23,0.153
3,Friday,165,23,0.139
4,Thursday,148,20,0.135
5,Wednesday,121,16,0.132
6,Tuesday,154,19,0.123


## Insight: Accident severity by day of week

This analysis examines how accident severity varies across the days of the week.
Rather than focusing only on accident volume, the proportion of serious and fatal accidents is calculated to identify days associated with higher-risk outcomes.


In [42]:
pd.read_sql_query("""
SELECT
  Weekday_or_Weekend,
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) AS severe_incidents,
  ROUND(
    1.0 * SUM(CASE WHEN "Accident Severity" IN ('Serious','Fatal') THEN 1 ELSE 0 END) / COUNT(*),
    3
  ) AS severe_rate
FROM Table3
GROUP BY Weekday_or_Weekend
ORDER BY severe_rate DESC;
""", conn)


,Weekday_or_Weekend,total_incidents,severe_incidents,severe_rate
0,Weekend,261,60,0.230
1,Weekday,738,101,0.137


**Insight:**

Weekend Vs Weekday severity
This analysis looks at the total incidents for Weekends Vs Weekdays.
Weekdays have more incidents but weekends have a higher severe rate.